<a href="https://www.kaggle.com/code/romanvelichkin/blue-book-for-bulldozers-how-to-get-into-top-25?scriptVersionId=142824922" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Blue Book for Bulldozers
Predict the auction sale price for a piece of heavy equipment to create a "blue book" for bulldozers.

The goal of the contest is to predict the sale price of a particular piece of heavy equiment at auction based on it's usage, equipment type, and configuaration.  The data is sourced from auction result postings and includes information on usage and equipment configurations.

https://www.kaggle.com/competitions/bluebook-for-bulldozers/

## Abstract

Here step by step you will learn how to get RMSLE score 0.2415 (top #24) using Random Forest Regressor.  
You will be able improve it even further - to 0.2412 (top #23) and probably more.

### Evaluation
The evaluation metric for this competition is the RMSLE (root mean squared log error) between the actual and predicted auction prices.

Sample submission files can be downloaded from the data page. Submission files should be formatted as follows:

Have a header: "SalesID,SalePrice";  
Contain two columns:
- SalesID: SalesID for the validation set in sorted order;
- SalePrice: Your predicted price of the sale.

## Prepare tools

In [ ]:
# import exploratory data analysis and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# models from scikit-learn
from sklearn.ensemble import RandomForestRegressor

# model evaluations
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

## Inspect data

In [ ]:
# Look what files do we have
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Importing training and validation sets
df = pd.read_csv("/kaggle/input/bluebook-for-bulldozers/TrainAndValid.csv", 
                 low_memory=False)
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

There are lot of missing values.

In [ ]:
df.columns

#### Let's look how sales and prices look on timeline

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df["saledate"][:1000], df["SalePrice"][:1000]);

We can't build proper plot because `saledate` is a string-type field. We need to parse dates from it first. We'll do it in next chapter.

In [ ]:
# Sales prices distribution
df.SalePrice.plot.hist();

In [ ]:
# Sales distribution over states
df.state.value_counts()

## Preparing data

### Parsing dates
Get data information from string column and turn it into data format that `pandas` understands. Use `parse_dates` parameter for that.

In [ ]:
# Import data again but this time parse dates
df = pd.read_csv("/kaggle/input/bluebook-for-bulldozers/TrainAndValid.csv",
                 low_memory=False,
                 parse_dates=["saledate"])

In [ ]:
df.saledate[:1000]

#### Building sales and prices scatter-plot on timeline again

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df["saledate"][:1000], df["SalePrice"][:1000]);

In [ ]:
# Look on data again. 
# There are too many columns so I turned dataframe to 90 degrees. 
df.head().T

### Sort dataframe by "saledate"
When working with time series data, it's a good idea to sort it by date.

In [ ]:
# Sort dataframe in date order
df.sort_values(by=["saledate"], inplace=True, ascending=True)
df.saledate.head()

### Make copy of dataframe
We make a copy of the original dataframe so when manipulate the copy, we'll keep original data.

In [ ]:
# Make a copy
df_tmp = df.copy()
df_tmp

### Add datetime parameters for `saledate` column
When we work with time-series data, we want to enrich the time & date component as much as possible.

In [ ]:
df_tmp["saleYear"] = df_tmp.saledate.dt.year
df_tmp["saleMonth"] = df_tmp.saledate.dt.month
df_tmp["saleDay"] = df_tmp.saledate.dt.day
df_tmp["saleDayOfWeek"] = df_tmp.saledate.dt.dayofweek
df_tmp["saleDayOfYear"] = df_tmp.saledate.dt.dayofyear

Now when we enriched our dataframe with date time features we can drop column "saledate"

In [ ]:
# Drop column "saledate"
df_tmp.drop("saledate", axis=1, inplace=True)

## Fill missing values
### Fill missing numeric values

In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        print(label)

In [ ]:
# Check for which numeric columns have null values

for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).any():
            print(label)

In [ ]:
# Fill numeric values with median

for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).any():
            # Add binary column which tells us if data was missing or not
            df_tmp[label+"_is_missing"] = pd.isnull(content)
            # Fill missing numeric values with median
            df_tmp[label] = content.fillna(content.median())

In [ ]:
# Check if there's still any null numeric values

for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).any():
            print(label)

No more nulls in numeric type values.

In [ ]:
# Check how many examples were missing
df_tmp.auctioneerID_is_missing.value_counts()

In [ ]:
df_tmp.isna().sum()

There are still null values in dataframe.

### Filling and turning missing categorical variables into numbers

In [ ]:
# Check for columns which aren't numeric
for label, content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

In [ ]:
# How will look state values after turning into numbers
pd.Categorical(df_tmp["state"]).codes

In [ ]:
# How will look missing values after turning into numbers
pd.Categorical(df_tmp["UsageBand"]).codes

We will increase all turned values by 1, so missing values will become 0.

In [ ]:
# Turn categorical variables into numbers and fill missing

for label, content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        # Add binart column to indicate whether sample had missing value
        df_tmp[label+"_is_missing"] = pd.isnull(content)
        # Turn all categories into numbers
        # Null values will be turned into "-1" so we add +1 to all numbers
        df_tmp[label] = pd.Categorical(content).codes + 1

In [ ]:
# Let's look result
df_tmp.head().T

In [ ]:
# Check if there are still any null values
df_tmp.isna().sum().sort_values(
    ascending=False)

Dataframe has no null values now.

## Modelling

We will use Random Forest Regressor. Because it's a good method to solve regression problems.

In [ ]:
%%time
# Let's test what's RFC performance on given data

# Instantiate model
model = RandomForestRegressor(n_jobs=-1,
                              random_state=42)

# Fit the model
model.fit(df_tmp.drop("SalePrice", axis=1), df_tmp["SalePrice"])

In [ ]:
# Score the model on initial data
model.score(df_tmp.drop("SalePrice", axis=1), df_tmp["SalePrice"])

### Splitting data into train/validations sets

In [ ]:
df_tmp.saleYear

In [ ]:
# Split data into training and validation sets
# Data from 2012 will be used for validation
df_val = df_tmp[df_tmp.saleYear == 2012]
df_train = df_tmp[df_tmp.saleYear != 2012]

len(df_train), len(df_val)

In [ ]:
# Split train and validation sets into X & y
X_train, y_train = df_train.drop("SalePrice", axis=1), df_train.SalePrice
X_val, y_val = df_val.drop("SalePrice", axis=1), df_val.SalePrice

X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
y_train

### Building an evaluation function

In [ ]:
# Create evaluation function (the competition uses RMSLE)
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score

def rmsle(y, y_preds):
    """
    Calculates root mean squared log error between predictions and true labels.
    """
    return np.sqrt(mean_squared_log_error(y, y_preds))

# Create function to evaluate model using different metrics
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_val)
    scores = {"training MAE": mean_absolute_error(y_train, train_preds),
              "validation MAE": mean_absolute_error(y_val, val_preds),
              "training RMSLE": rmsle(y_train, train_preds),
              "validation RMSLE": rmsle(y_val, val_preds),
              "training R^2": r2_score(y_train, train_preds),
              "validation R^2": r2_score(y_val, val_preds)
             }
    return scores

### Testing our model on a subset

In [ ]:
# Change max samples value
model = RandomForestRegressor(n_jobs=-1,
                              random_state=42,
                              max_samples=10000)

In [ ]:
%%time
model.fit(X_train, y_train)

In [ ]:
show_scores(model)

Even without tuning Random Forest Regressor gives us result that could get at least Bronze medal. 

### Hyperparameters tuning with RandomizedSearchCV

In [ ]:
%%time

# Different RandomForestRegressor hyperparameters
rf_grid = {"n_estimators": [30, 40, 50],
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": [10, 14, 18],
           "min_samples_leaf": [1, 2, 3],
           "max_features": [0.5, 1, "sqrt", "auto"],
           "max_samples": [10000]}

# Instantiate RandomiedSearchCV model
rs_model = RandomizedSearchCV(RandomForestRegressor(n_jobs=-1,
                                                   random_state=42),
                                                   param_distributions=rf_grid,
                                                   n_iter=10,
                                                   cv=5,
                                                   verbose=True)

rs_model.fit(X_train, y_train)

In [ ]:
# Find the best model hyperparameters
rs_model.best_params_

In [ ]:
# Evaluate the RandomizedSearh model
show_scores(rs_model)

### Hyperparameters tuning with GridSearchCV

In [ ]:
%%time

# Different RandomForestRegressor hyperparameters
rf_grid = {"n_estimators": [170, 200],
           "min_samples_split": [4, 5],
           "min_samples_leaf": [2],
           "max_features": [0.4, 0.45, 0.5],
           "max_samples": [10000]}

# Instantiate GridSearchCV model
gs_model = GridSearchCV(RandomForestRegressor(n_jobs=-1,
                                              random_state=42),
                                              param_grid=rf_grid,
                                              cv=5,
                                              verbose=True)

gs_model.fit(X_train, y_train)

In [ ]:
# Find the best model hyperparameters
gs_model.best_params_

In [ ]:
# Evaluate the GridSearh model
show_scores(gs_model)

### Train a model with the best hyperparameters
I've tuned hyperparameters by hand and improved final result.

In [ ]:
%%time

# I've tuned hyperparameters by hand a bit
# Best hyperparameters
best_model = RandomForestRegressor(min_samples_leaf=2,
                                   min_samples_split=5,
                                   n_estimators=200,
                                   max_features=0.45,
                                   n_jobs=-1,
                                   max_samples=None,
                                   random_state=42)

#Fit the model with those hyperparameters
best_model.fit(X_train, y_train)

You can try to increase `n_estimators`. At some range it will improve score. However it will slow down training model drastically.

`n_estimators=500` gave me RMSLE = 0.2414  
`n_estimators=5000` gave me RMSLE = 0.2412, but it took an hour to train model on my PC. Training it here on Kaggle could take times more.


In [ ]:
# Scores for best model trained on all the data
show_scores(best_model)

## Make predictions on test data

In [ ]:
# Import the test data
df_test = pd.read_csv("/kaggle/input/bluebook-for-bulldozers/Test.csv",
                      low_memory=False,
                      parse_dates=["saledate"])
df_test.head()

In [ ]:
df_test.isna().sum()

Test data has nulls so we need to deal with it.

### Preprocessing test data (getting it into same format as training dataset)

In [ ]:
def preprocess_data(df):
    """
    Performs transformation on df and returns transformed df
    """
    # Enrich dataframe with date time features
    df["saleYear"] = df.saledate.dt.year
    df["saleMonth"] = df.saledate.dt.month
    df["saleDay"] = df.saledate.dt.day
    df["saleDayOfWeek"] = df.saledate.dt.dayofweek
    df["saleDayOfYear"] = df.saledate.dt.dayofyear

    # Now when we enriched our dataframe with date time features we can drop column "saledate"
    df.drop("saledate", axis=1, inplace=True)

    # Fill numeric values with median
    for label, content in df.items():
        if pd.api.types.is_numeric_dtype(content):
            if pd.isnull(content).sum():
                # Add binary column which tells us if data was misasing or not
                df[label+"_is_missing"] = pd.isnull(content)
                # Fill missing numeric values with median
                df[label] = content.fillna(content.median())

    # Turn categorical variables into numbers and fill missing
        if not pd.api.types.is_numeric_dtype(content):
            # Add binart column to indicate whether sample had missing value
            df[label+"_is_missing"] = pd.isnull(content)
            # Turn missing categories into number and add +1
            df[label] = pd.Categorical(content).codes + 1
            
    return df

In [ ]:
df_test = preprocess_data(df_test)
df_test.head().T

In test dataset doesn't have nulls in `auctioneerID` column. As result preprocessed test dataset won't have `auctioneerID_is_missing` column. 

In [ ]:
# We can find how the columns differ using sets
set(X_train.columns) - set(df_test.columns)

Manually adjust df_test to have `auctioneerID_is_missing` column.


In [ ]:
# Adding auctioneerID_is_missing column
df_test["auctioneerID_is_missing"] = False
df_test.head()

### Making predictions on test data

In [ ]:
# Make predictions on test data
test_preds = best_model.predict(df_test)

test_preds

In [ ]:
# Format prediction into the same format Kaggle is after
df_preds = pd.DataFrame()
df_preds["SalesID"] = df_test["SalesID"]
df_preds["SalePrice"] = test_preds
df_preds

If you need, you can export predictions.

In [ ]:
# Export prediction data
# You can uncomment it
# df_preds.to_csv("/kaggle/input/bluebook-for-bulldozers/test_prediction.csv", index=False)

### Feature importance

In [ ]:
# Find feature importance of best model
best_model.feature_importances_

In [ ]:
# Create dataframe with feature importances
features = (pd.DataFrame({"features": df_test.columns,
                          "feature_importances": best_model.feature_importances_})
            .sort_values("feature_importances", ascending=False)
            .reset_index(drop=True))

features

In [ ]:
# Plot the dataframe
fig, ax = plt.subplots(figsize=(10,20))
ax.barh(features["features"][:50], features["feature_importances"][:50])
ax.set_ylabel("Features")
ax.set_xlabel("Feature importance");

## Result

We've improved our data by enriching it with date and time features. We also dealt with `null` values.
As result we trained model with RMSLE score 0.2415 (top #24) and found way to improve it further - to 0.2412 (top #23).

Also we ranked features by its importance. `YearMade` and `ProductSize` are by far most important features.